# Resnet50の転移学習で画像分類

## 学習

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
from pathlib import Path

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets.utils import download_url
#from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100

#data_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/"
#class_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/class_name/"
CIFAR100_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/CIFAR100/"
#CIFAR10_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/CIFAR10/"

epochs = 20
batch_size = 256 
#性能改善対策
# バッチサイズでよく見かけるのは、、1,32,128,256,516
#    →1だと完全に確率的勾配降下法になる。
#    →バッチサイズを上げると学習速度は速くなる。
# lossの揺れ幅が各epochで大きかったら、ミニバッチのサイズを上げる
# 揺れ幅が小さくて、データのサイズも大きくない場合は、ミニバッチのサイズを下げる
#試したこと
#　①エポック2、バッチサイズ16 →ロスが1を切らない切らない。エポックだけあげたいが時間かかるのでバッチサイズも上げる。
#  ②エポック10、バッチサイズ128 →ロスが1を切らない切らない。エポックだけあげたいが時間かかるのでバッチサイズも上げる。
#  ③エポック20、バッチサイズ128 →ロスは3.1あたりからスタート。ロスが1を切らない切らない。エポックだけあげたいが時間かかるのでバッチサイズも上げる。
#  ④エポック60、バッチサイズ512 →ロスは1.6あたりからスタート。ロスが数回で0近くになる。エポックも下げバッチサイズも下げる。
#  ⑤エポック20、バッチサイズ256 →ロスは2.8からスタート。ギリギリロス1を切るぐらいなのでエポック数あげるあげる
#  ⑥エポック40、バッチサイズ256　→イテレーションイテレーション26回でロス1。GPUの上限に到達して切断。
#  ⑦エポック50、バッチサイズ128　→transforms.Composeを修正して明かに学習不足学習不足20回回しても回しても2を切らないし、一部上がったり下がったりしている。
#  ⑧エポック50、バッチサイズ32　→transforms.Composeを修正して明かに学習不足学習不足30回あたりから2.4を切らないし、一部上がったり下がったりしている。
#  ⑨エポック70、バッチサイズ128　→transforms.Composeを修正後。3.8始まりで1.7ぐらいまで行くので、学習不足のためエポック数上げる。
#  ⑩エポック20、バッチサイズ256　→transforms.Composeを修正したら精度悪くなる。前処理が入り処理が遅くなるのでエポック数下げる。

# デバイス設定
def get_device(use_gpu):
    if use_gpu and torch.cuda.is_available():
        print('GPUを使用して計算します')
        # これを有効にしないと、計算した勾配が毎回異なり、再現性が担保できない。
        torch.backends.cudnn.deterministic = True
        return torch.device("cuda")
    else:
        print('CPUを使用して計算します')
        return torch.device("cpu")

# デバイスを選択する。
device = get_device(use_gpu=True)

# Resnetモデル
model = torchvision.models.resnet50(pretrained=True)

# 勾配の計算をしないようにする。
for param in model.parameters():
    param.requires_grad = False

# モデルの最終層を100個のクラスの予測用に改良する。
model.fc = nn.Linear(2048, 100)

# GPU転送
model = model.to(device)

#訓練するCIFAR100の画像サイズは32*32で均一。
#前処理してresnetのinputサイズ（224×224）に合わせないと精度が悪くなる。
transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。　
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

# 学習モード設定
model.train()

# Dataset を作成する。
#dataset = CIFAR10(root=CIFAR10_path,
dataset = CIFAR100(root=CIFAR100_path,
                   train=True,
                   download=True,
                   transform=transform)
               
# DataLoader を作成する。
dataloader = DataLoader(dataset, 
                        batch_size=batch_size,
                        shuffle=True)

# 損失関数とオプティマイザー
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 訓練ループ
for epoch in range(epochs):

    #ロス
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(dataloader):

        # 勾配(グラディエント)をゼロにする
        optimizer.zero_grad()

        #inputs = batch["image"].to(device)
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        # 損失関数でロスを計算し、逆伝播する
        loss = criterion(outputs, labels)

        # バックプロップ（勾配計算）
        loss.backward()

        # モデルの最適化調整
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # print("学習：",i)
        # 200回に一回、平均のロスを表示
        # if (i+1) % 200 == 0:
        # if (i+1) % 95 == 0: #バッチサイズサイズ512の時は97回で1イテレーション終了するためするため95回に1回ログ出力する。
        if (i+1) % 200 == 0:
            print(f'[{epoch+1}, {i+1:5d}] loss: {running_loss/200:.3f}')
            running_loss = 0.0

print("学習終了")
# モデルの中身確認
#print(model.state_dict())

# GPUで学習したモデルの保存
save_gpu_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/trained_model/tl_resnet50_gpu.pth"
torch.save(model.state_dict(), save_gpu_path)

# 念の為念の為CPUに変更したものも保存
save_cpu_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/trained_model/tl_resnet50_cpu.pth"
torch.save(model.to('cpu').state_dict(), save_cpu_path)       
print("保存終了")

In [ ]:
#損失をグラフ化化
fig = plt.figure()
plt.xlabel('epoch')
plt.plot(epoch, running_loss, label='train_loss')
plt.legend()
plt.show()

## 精度検証

In [ ]:
import json
from pathlib import Path

import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets.utils import download_url
#from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100

#data_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/"
#class_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/class_name/"
CIFAR100_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/CIFAR100/"
#CIFAR10_path = "/content/drive/My Drive/Colab Notebooks/resnet_app/data/CIFAR10/"

epochs = 20
batch_size = 256

# デバイス設定
def get_device(use_gpu):
    if use_gpu and torch.cuda.is_available():
        print('GPUを使用して計算します')
        # これを有効にしないと、計算した勾配が毎回異なり、再現性が担保できない。
        torch.backends.cudnn.deterministic = True
        return torch.device("cuda")
    else:
        return torch.device("cpu")

# デバイスを選択する。
device = get_device(use_gpu=True)

# Resnetモデル
model = torchvision.models.resnet50(pretrained=True)

# モデルの最終層を100個のクラスの予測用に改良する。
model.fc = nn.Linear(2048, 100)

# 学習データ読み込み
model.load_state_dict(torch.load(save_gpu_path))

# モデルを評価モードにする
model.eval()

# GPU転送
model = model.to(device)

#transforms.Composeは、複数のTransform（例：リサイズや中心と切り取るなど）を連続して行う Transform を作成する。
# Transform はデータに対して行う前処理を定義。
# torchvisionでは、画像のリサイズや切り抜きといった処理を行うためのTransformが用意されてる。
#　→transforms.Composeの引数について
# 　　https://pystyle.info/pytorch-list-of-transforms/#outline__5_2
# resnet系はImagenetの画像224×224で学習しているので、入力画像を画像を224にする。
transform = transforms.Compose(
    [
        transforms.Resize(256),  # (256, 256) で切り抜く。
        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
        transforms.ToTensor(),  # テンソルにする。
        transforms.Normalize( # 標準化する。
            mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
        ),  
    ]
)

# Dataset を作成する。
testset = CIFAR100(root=CIFAR100_path,
                   train=False,
                   download=True,
                   transform=transform)
               
# DataLoader を作成する。
dataloader = DataLoader(testset, 
                        batch_size=batch_size,
                        shuffle=False)

# 正解と合計の数
correct = 0
total = 0

for i, (inputs, labels) in enumerate(dataloader):
    # 予測値
    with torch.no_grad():
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)

    # クラスの予測へと変換
    prediction = outputs.argmax(axis=1)

    # 正解の数を数える
    correct += (labels==prediction).sum().item()
    total += len(labels)

print(f'テスト終了: accuracy={correct/total*100:.2f}')